In [127]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

import types
from botocore.client import Config
import ibm_boto3

from  geopy.geocoders import Nominatim
import time

from pandas.io.json import json_normalize

import warnings
warnings.filterwarnings('ignore')

#!pip install folium
import folium
from folium.plugins import HeatMap

import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_81e1207262474217882b49886b0c06fb = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='COTH85J9RgBEfOxpY-QMDURR9yVxYOhJgUk-PZlfoPeK',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_81e1207262474217882b49886b0c06fb.get_object(Bucket='indianrestaurantdatascienceprojec-donotdelete-pr-s7ddpphnx3qfml',Key='Indians in US.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()

,State,Indian population
0,California,"528,120"
1,New York,"313,620"
2,New Jersey,"292,256"
3,Texas,"245,981"
4,Illinois,"188,328"


In [5]:
#To convert the feature "Indian population to int32, we have to remove the comma(,) character from each numbers"
population = []
for val in data['Indian population'].values:
    population.append(int(str(val).replace(',',''))) #Replace all comma(,)

In [6]:
#creating Indian population as type 'int32' after dropping the same column
data.drop(['Indian population'], axis = 1, inplace = True) 
data['Indian population'] = population

In [7]:
data.head()

,State,Indian population
0,California,528120
1,New York,313620
2,New Jersey,292256
3,Texas,245981
4,Illinois,188328


### Finding Latitude and longitudes of the Areas

In [16]:
#The geocoder API have some call rate limit. so for calling a bulk time we have to add a sleep of 1.25 sec to avoid service denial.
def find_lat_lng(df):
    geolocator = Nominatim()

    if type(df) == str: #check whether the data is a single string or a pandas DataFrame
        loc = geolocator.geocode(df) #calling geocoder api
        return loc.latitude, loc.longitude
    else:
        lat = []
        lng = []
        for place in df[df.columns[0]].values:
            try:
                loc = geolocator.geocode(str(place)) #calling geocoder api
                lat.append(loc.latitude)
                lng.append(loc.longitude)
            except Exception as e:
                print("Trying again")

            time.sleep(1.25) 
        return lat, lng

In [9]:
#retriving list of latitude and longitude using geocoder api
data['Latitute'], data['Longitude'] = find_lat_lng(data)

In [10]:
data.head()

,State,Indian population,Latitute,Longitude
0,California,528120,36.701463,-118.755997
1,New York,313620,40.712728,-74.006015
2,New Jersey,292256,40.075738,-74.404162
3,Texas,245981,31.816038,-99.512099
4,Illinois,188328,40.079661,-89.433729


In [20]:
#Function to generate Folium map
def generateBaseMap(default_zoom_start=3, default_location=[48, -102]):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start, )
    return base_map

In [24]:
us_lat, us_lng = find_lat_lng('US')
us_map = generateBaseMap(default_zoom_start = 5, default_location = [us_lat, us_lng] )
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=data,
    columns=['State', 'Indian population'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Indian Population'
).add_to(us_map)
folium.LayerControl().add_to(us_map)
us_map

### Getting the html document of the link using requests method¶

In [25]:
data_url = requests.get("https://en.wikipedia.org/wiki/Indian_Americans", verify = False).text
data_url

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Indian Americans - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XlSnpQpAMFUAALbPiRkAAACI","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Indian_Americans","wgTitle":"Indian Americans","wgCurRevisionId":942507182,"wgRevisionId":942507182,"wgArticleId":18963843,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","All articles with dead external 

### Parsing through the html document using BeautifulSoap

In [26]:
soup = BeautifulSoup(data_url, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Indian Americans - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XlSnpQpAMFUAALbPiRkAAACI","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Indian_Americans","wgTitle":"Indian Americans","wgCurRevisionId":942507182,"wgRevisionId":942507182,"wgArticleId":18963843,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","All articles with de

### finding the table tag and curresponding data

In [27]:
table = soup.find('table', {'class', 'wikitable sortable'})
table

<table class="wikitable sortable" style="text-align: right;">
<caption>Asian Indian population in <a class="mw-redirect" href="/wiki/Metropolitan_Statistical_Area" title="Metropolitan Statistical Area">Metropolitan Statistical Areas</a> of the United States of America
</caption>
<tbody><tr>
<th>Metropolitan Statistical Area</th>
<th data-sort-type="numeric">Indian American<br/>population (2010)<sup class="reference" id="cite_ref-proximityone.com_37-0"><a href="#cite_note-proximityone.com-37">[37]</a></sup></th>
<th data-sort-type="numeric">Total population (2010)</th>
<th data-sort-type="numeric">% of Total<br/>population</th>
<th>Combined Statistical Area
</th></tr>
<tr>
<td style="text-align:left;"><a href="/wiki/New_York_metropolitan_area" title="New York metropolitan area">New York–Newark–Jersey City, NY–NJ–PA</a></td>
<td>526,133</td>
<td>18,897,109</td>
<td>2.8%</td>
<td style="text-align:left;"><a class="mw-redirect" href="/wiki/New_York-Newark,_NY-NJ-CT-PA_Combined_Statistical_

In [28]:
content = table.findAll('tr')
city = [] #to store the metropolitian city
area = []
indian = [] #to store population of indians
population = [] # to store total population
for data in content[1:]:
    city.append(str(data.findAll('td')[0].get_text()).replace('–','-').split('-')[0])
    area.append(str(data.findAll('td')[0].get_text()).replace('–','-'))
    indian.append(int(data.findAll('td')[1].get_text().replace(',','')))
    population.append(int(data.findAll('td')[2].get_text().replace(',','')))

In [29]:
df = pd.DataFrame()
df['Cities'] = city
df['Metropolitan areas'] = area
df['Indian population'] = indian
df['Total population'] = population

df.head()

,Cities,Metropolitan areas,Indian population,Total population
0,New York,"New York-Newark-Jersey City, NY-NJ-PA",526133,18897109
1,Chicago,"Chicago-Naperville-Elgin, IL-IN-WI",171901,9461105
2,Washington,"Washington-Arlington-Alexandria, DC-VA-MD-WV",127963,5582170
3,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",119901,12828837
4,San Francisco,"San Francisco-Oakland-Hayward, CA",119854,4335391


In [30]:
df.shape

(40, 4)

In [31]:
df['Latitute'], df['Longitude'] = find_lat_lng(df)

In [32]:
df.head()

,Cities,Metropolitan areas,Indian population,Total population,Latitute,Longitude
0,New York,"New York-Newark-Jersey City, NY-NJ-PA",526133,18897109,40.712728,-74.006015
1,Chicago,"Chicago-Naperville-Elgin, IL-IN-WI",171901,9461105,41.875562,-87.624421
2,Washington,"Washington-Arlington-Alexandria, DC-VA-MD-WV",127963,5582170,38.894893,-77.036553
3,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",119901,12828837,34.053691,-118.242767
4,San Francisco,"San Francisco-Oakland-Hayward, CA",119854,4335391,37.779026,-122.419906


In [33]:
df.describe()

,Indian population,Total population,Latitute,Longitude
count,40.000000,4.000000e+01,40.000000,40.000000
mean,55745.850000,3.668441e+06,37.966803,-91.785742
std,86988.631609,3.506253e+06,5.365692,16.289656
min,3534.000000,3.665130e+05,25.774266,-122.674195
25%,15011.750000,1.746253e+06,33.977543,-98.170275
50%,23556.500000,2.626986e+06,39.100779,-86.891386
75%,66693.500000,4.389644e+06,41.251571,-79.652339
max,526133.000000,1.889711e+07,49.583333,-71.058291


In [39]:
base_map = generateBaseMap(default_zoom_start = 5.1, default_location = [us_lat, us_lng])
HeatMap(data=df[['Latitute', 'Longitude', 'Indian population']], radius=13, max_zoom=13).add_to(base_map)
base_map

### Preferred states for Indian restuarant - New York

**As New York is having high Indian population, we will consider to Segment and Cluster the cities of New york.**

In [40]:
df[df['Metropolitan areas'].str.contains("NY")]

,Cities,Metropolitan areas,Indian population,Total population,Latitute,Longitude
0,New York,"New York-Newark-Jersey City, NY-NJ-PA",526133,18897109,40.712728,-74.006015


**From the above table, indian population is very high in Los angeles.**

# Analysis in New York

In [41]:

body = client_81e1207262474217882b49886b0c06fb.get_object(Bucket='indianrestaurantdatascienceprojec-donotdelete-pr-s7ddpphnx3qfml',Key='NY_neibours.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_NY = pd.read_csv(body)
df_data_NY.drop(['Unnamed: 0'], axis = 1, inplace = True)
df_data_NY.head()


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [42]:
df_data_NY['Borough'].value_counts()

Queens           81
Brooklyn         70
Staten Island    63
Bronx            52
Manhattan        40
Name: Borough, dtype: int64

#### In New York city, Queens have the largest population of Indians

[Source](https://en.wikipedia.org/wiki/Indians_in_the_New_York_City_metropolitan_region)

In [43]:
df_queens = df_data_NY[df_data_NY['Borough'].str.contains("Queens")]
df_queens.head()

,Borough,Neighborhood,Latitude,Longitude
129,Queens,Astoria,40.768509,-73.915654
130,Queens,Woodside,40.746349,-73.901842
131,Queens,Jackson Heights,40.751981,-73.882821
132,Queens,Elmhurst,40.744049,-73.881656
133,Queens,Howard Beach,40.654225,-73.838138


In [44]:
df_queens.reset_index(drop = True, inplace = True)
df_queens.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [45]:
df_queens.shape

(81, 4)

#### Get Latitude and Longitude of Queens

In [46]:
lat_queen, lng_queen = find_lat_lng('Queen, New York, US')
print(lat, lng)

39.7837304 -100.4458825


In [50]:
ny_map = generateBaseMap(default_zoom_start = 11, default_location = [lat_queen, lng_queen])

for lat, lng, label in zip(df_queens['Latitude'], df_queens['Longitude'], df_queens['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(ny_map)
    
ny_map

### Explose the Neighborhoods using Four Square API 

In [95]:
CLIENT_ID = '5M40ULMEDEGPYJBQN3Z0RFOU3VZEG1FVHFHF5TP2OVJGVTLU' # your Foursquare ID
CLIENT_SECRET = 'HGUYAXIGANKU3JF3IIIEYV30FDQXS4EB10FTNYKA3HFXOQ2W' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
search_query = 'India'
radius = 500
LIMIT = 100

In [96]:
#find the first Neighborhood
print(f"Neighborhood : {df_queens['Neighborhood'][0]}\nLatitude : {df_queens['Latitude'][0]}\nLongitude : {df_queens['Longitude'][0]} ")

Neighborhood : Astoria
Latitude : 40.76850859335492
Longitude : -73.91565374304234 


In [97]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df_queens['Latitude'][0], 
    df_queens['Longitude'][0], 
    radius, 
    LIMIT)

In [98]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e54c3a10f596835c343bd27'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Astoria',
  'headerFullLocation': 'Astoria, Queens',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 107,
  'suggestedBounds': {'ne': {'lat': 40.773008597854925,
    'lng': -73.90972309237958},
   'sw': {'lat': 40.76400858885492, 'lng': -73.9215843937051}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bdf502a89ca76b062b75d5e',
       'name': 'Favela Grill',
       'location': {'address': '33-18 28th Ave',
        'crossStreet': 'btwn 33rd & 34th St.',
        'lat': 40.76734843380796,
        'lng': -73.9178974

In [99]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [100]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Favela Grill,Brazilian Restaurant,40.767348,-73.917897
1,Orange Blossom,Gourmet Shop,40.769856,-73.917012
2,Titan Foods Inc.,Gourmet Shop,40.769198,-73.919253
3,Simply Fit Astoria,Gym,40.769114,-73.912403
4,CrossFit Queens,Gym,40.769404,-73.918977


In [101]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Queens

#### Let's create a function to repeat the same process to all the neighborhoods in Queens

In [102]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [103]:
queens_venue = getNearbyVenues(names=df_queens['Neighborhood'],
                                   latitudes=df_queens['Latitude'],
                                   longitudes=df_queens['Longitude']
                                  )

Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


In [105]:
print(queens_venue.shape)
queens_venue.head()

(2100, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym
4,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym


Let's check how many venues were returned for each neighborhood

In [107]:
queens_venue.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,17,17,17,17,17,17
Astoria,100,100,100,100,100,100
Astoria Heights,14,14,14,14,14,14
Auburndale,17,17,17,17,17,17
Bay Terrace,37,37,37,37,37,37
Bayside,71,71,71,71,71,71
Bayswater,2,2,2,2,2,2
Beechhurst,15,15,15,15,15,15
Bellaire,13,13,13,13,13,13


#### Let's find out how many unique categories can be curated from all the returned venues

In [108]:
print('There are {} uniques categories.'.format(len(queens_venue['Venue Category'].unique())))

There are 267 uniques categories.


## 3. Analyze Each Neighborhood

In [109]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venue[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venue['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queens_onehot.columns[-1]] + list(queens_onehot.columns[:-1])
queens_onehot = queens_onehot[fixed_columns]

queens_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [111]:
queens_onehot.shape

(2100, 267)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [112]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Arverne,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [113]:
queens_grouped.shape

(81, 267)

#### Let's print each neighborhood along with the top 5 most common venues

In [115]:
num_top_venues = 5

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arverne----
            venue  freq
0       Surf Spot  0.24
1   Metro Station  0.12
2  Sandwich Place  0.12
3     Coffee Shop  0.06
4       Wine Shop  0.06


----Astoria----
                       venue  freq
0                        Bar  0.07
1  Middle Eastern Restaurant  0.06
2           Greek Restaurant  0.05
3                 Hookah Bar  0.05
4         Seafood Restaurant  0.04


----Astoria Heights----
           venue  freq
0  Deli / Bodega  0.07
1    Pizza Place  0.07
2    Bus Station  0.07
3  Bowling Alley  0.07
4    Supermarket  0.07


----Auburndale----
               venue  freq
0  Mobile Phone Shop  0.06
1         Hookah Bar  0.06
2   Sushi Restaurant  0.06
3     Gymnastics Gym  0.06
4       Noodle House  0.06


----Bay Terrace----
               venue  freq
0     Clothing Store  0.11
1      Women's Store  0.05
2         Kids Store  0.05
3  Mobile Phone Shop  0.05
4     Lingerie Store  0.05


----Bayside----
                 venue  freq
0                  Bar  0.07
1  Am

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [116]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [117]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Sandwich Place,Metro Station,Bus Stop,Thai Restaurant,Donut Shop,Beach,Coffee Shop,Playground,Board Shop
1,Astoria,Bar,Middle Eastern Restaurant,Greek Restaurant,Hookah Bar,Seafood Restaurant,Pizza Place,Mediterranean Restaurant,Dessert Shop,Salon / Barbershop,Café
2,Astoria Heights,Hostel,Bakery,Plaza,Deli / Bodega,Chinese Restaurant,Shopping Mall,Bus Station,Burger Joint,Food,Pizza Place
3,Auburndale,Miscellaneous Shop,Gymnastics Gym,Noodle House,Korean Restaurant,Supermarket,Furniture / Home Store,Italian Restaurant,Discount Store,Athletics & Sports,Toy / Game Store
4,Bay Terrace,Clothing Store,Women's Store,Donut Shop,Kids Store,Lingerie Store,Shoe Store,Mobile Phone Shop,American Restaurant,Cosmetics Shop,Coffee Shop


## 4. Cluster Neighborhoods

In [118]:
from sklearn.cluster import KMeans

In [121]:
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [125]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

queens_merged = df_queens

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,1,Bar,Middle Eastern Restaurant,Greek Restaurant,Hookah Bar,Seafood Restaurant,Pizza Place,Mediterranean Restaurant,Dessert Shop,Salon / Barbershop,Café
1,Queens,Woodside,40.746349,-73.901842,1,Grocery Store,Thai Restaurant,Bakery,Filipino Restaurant,Latin American Restaurant,Donut Shop,Pizza Place,Bar,American Restaurant,Pub
2,Queens,Jackson Heights,40.751981,-73.882821,1,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mexican Restaurant,Mobile Phone Shop,Thai Restaurant,Diner,Spanish Restaurant,Pharmacy
3,Queens,Elmhurst,40.744049,-73.881656,1,Thai Restaurant,Mexican Restaurant,South American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Colombian Restaurant,Bubble Tea Shop,Snack Place,Food Court,Seafood Restaurant
4,Queens,Howard Beach,40.654225,-73.838138,1,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Pharmacy,Bagel Shop,Supermarket,Spa,Shipping Store,Mexican Restaurant,Seafood Restaurant


Finally, let's visualize the resulting clusters

In [128]:
map_clusters = folium.Map(location=[lat_queen, lng_queen], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# THANK YOY